# Pip Installs

In [1]:
# pip install tiktoken

In [2]:
# pip install pinecone

In [3]:
# pip install pymupdf

In [4]:
# pip install langchain_pinecone

In [5]:
# pip install langchain_openai

In [6]:
# pip install pytesseract pdf2image

# Code

In [2]:
import fitz
import os
import tiktoken
from dotenv import load_dotenv
import openai
from pinecone import Pinecone, ServerlessSpec

import langchain
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.schema import Document

c:\Users\antho\anaconda3\envs\AirScope\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Testing pdf loader

In [ ]:
class PDFLoader:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path

    def extract_text(self):
        doc = fitz.open(self.pdf_path)
        text = ""

        for page in doc:
            text += page.get_text()
        return text

In [4]:
load_dotenv()

class EmbeddingGenerator:
    def __init__(self):
        openai_api_key = os.getenv("OPENAI_API_KEY")
        if not openai_api_key:
            raise ValueError("OPENAI_API_KEY not set")
        
    # split into chunks by tokens
    def chunk_text_by_tokens(self, text, chunk_size, encoding_name="cl100k_base"):
        encoding = tiktoken.get_encoding(encoding_name)
        tokens = encoding.encode(text)
        return [encoding.decode(tokens[i:i + chunk_size]) for i in range(0, len(tokens), chunk_size)]

    # generate embeddings for each chunk and return list of embeddings
    def generate_embeddings(self, chunks):
        embeddings = []
        for chunk in chunks:
            response = openai.embeddings.create(
                input=chunk,
                model="text-embedding-ada-002"
            )
            embeddings.append(response.data[0].embedding)
        return embeddings
    
    # split text into chunks and generate embeddings
    def process_text(self, text, chunk_size=1000):
        if not isinstance(text, str) or len(text) == 0:
            raise ValueError("Input text must be a non-empty string")
        chunks = self.chunk_text_by_tokens(text, chunk_size)
        embeddings = self.generate_embeddings(chunks)
        return chunks, embeddings

# if __name__ == "__main__":
#     generator = EmbeddingGenerator()
#     chunks, embeddings = generator.process_text(text, chunk_size=800)

In [ ]:
class PineconeStore:

    def __init__(self, environmeent="us-east-1"):
        pinecone_api_key = os.getenv("PINECONE_API_KEY")
        if not pinecone_api_key:
            raise ValueError("PINECONE_API_KEY not set")
        
        # pinecone instance
        self.pc = Pinecone(ap_key=pinecone_api_key)
        self.index_name = "text-analyzer"

        if self.index_name not in self.pc.list_indexes().names():
            self.pc.create_index(
                name=self.index_name,
                dimension=1536,
                metric='cosine',
                spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
    
    def save_vectors(self, vectors, metadata, chunks):
        index = self.pc.Index(self.index_name)

        # save each embedding with unique metadata
        for i, vector in enumerate(vectors):
            vector_id = f"{metadata['id']}_chunk_{i}" # uniqe id
            chunk_metadata = {
                "id": vector_id,
                "source" : metadata["source"],
                "chunk" : i,
                "text": chunks[i]
            }

            index.upsert(vectors=[(vector_id, vector, chunk_metadata)])

    def delete(self):
        index = self.pc.Index(self.index_name)
        index.delete(delete_all=True)

# if __name__ == "__main__":
#     vector_store = PineconeStore()
#     vector_store.save_vectors(embeddings, {"id": "doc_1", "source": "example.pdf"}, chunks)

In [6]:
class PineconeRetriever:
    def __init__(self, pinecone_api_key, openai_api_key):

        # pinecone connection
        self.pc = Pinecone(api_key=pinecone_api_key)
        self.index_name = "text-analyzer"
        self.index = self.pc.Index(self.index_name)

        # openAi model and embeddings
        self.embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
        self.llm = OpenAI(temperature=0, api_key=openai_api_key)

        # create the Pinecone vector store
        self.vector_store = PineconeVectorStore(index=self.index, embedding=self.embedding_model, text_key="text")
        self.retriever = self.vector_store.as_retriever()

        # create the RetrievalQA chain
        self.qa_chain = RetrievalQA.from_chain_type(llm=self.llm, chain_type="stuff", retriever=self.retriever)

    def query(self, query_text):
        # execute the QA chain with the input query
        response = self.qa_chain.invoke({"query": query_text})
        return response['result']


In [ ]:
if __name__ == '__main__':

    folder_path = "./pdfs"
    generator = EmbeddingGenerator()
    vector_store = PineconeStore()

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".pdf"):
                pdf_path = os.path.join(folder_path, file)
                print(f"Processing: {file}")

                try:
                    loader = PDFLoader(pdf_path)
                    text = loader.extract_text()

                    chunks, embeddings = generator.process_text(text, chunk_size=800)
                    metadata = {"id": file.split(".")[0], "source": file}
                    vector_store.save_vectors(embeddings, metadata, chunks)

                except Exception as e:
                    print(f"Error processing {file}: {e}")

    pinecone_api_key = os.getenv("PINECONE_API_KEY")
    openai_api_key = os.getenv("OPENAI_API_KEY")

    retriever = PineconeRetriever(pinecone_api_key=pinecone_api_key, openai_api_key=openai_api_key)


    # query = ("You are a genius on everything related to civil engineering and conrete canoe."
    #          "You only answer questions based on this topic. Tell me about the Project Proposals."
    #           )

    # result = retriever.query(query)
    # print(result)

Processing: # 1 Universtié Laval - Maple Drifter - Project Proposal - 2024-05-13.pdf
Error processing # 1 Universtié Laval - Maple Drifter - Project Proposal - 2024-05-13.pdf: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 24 Feb 2025 19:22:25 GMT', 'Content-Type': 'application/json', 'Content-Length': '148', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '22', 'x-pinecone-request-id': '2569726938939669781', 'x-envoy-upstream-service-time': '7', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector ID must be ASCII, but got '# 1 Universtié Laval - Maple Drifter - Project Proposal - 2024-05-13_chunk_0'","details":[]}

Processing: #1 2019 UFCC Design Paper Official National Draft.pdf
Processing: #1 California Polytechnic State University, San Luis Obispo - Europa - Project Proposal - 2022.pdf
Processing: #1 California Polytechnic State University, San Luis Obispo - Oceana - Project Proposal - 2023.pdf
Processing: #1 TONGJI UNIVE

In [17]:
query = ("Tell me about the Concrete Canoe Project Proposals of NYU, what were the names of some of their canoes?"
              )

result = retriever.query(query)
print(result)

 Unfortunately, I do not have access to the specific names of the canoes proposed by NYU in their Concrete Canoe Project Proposals. However, some of the key components of their proposals included a hull design, structural analysis, mix design, construction process, project schedule, and sustainability plan.


In [ ]:
# vector_store.delete()